In [158]:
from espn_api.football import League, Player
import json
import espn_api
import os
from itertools import groupby
from collections import defaultdict
import espn_api.football as e
import requests
import pandas as pd

from typing import List, Dict
import pyperclip
import time
from objects import *
from utils import *

In [509]:
POSITIONS = ['WR', 'RB', 'TE', 'DE', 'LB', 'S', 'CB', 'DT', 'QB', 'K']

GAME_LOGS_LINK = "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/athletes/{player}/gamelog?region=us&lang=en&contentorigin=espn&season={year}"

In [3]:
LAMAR = 3916387
DICKER = 4362081

In [4]:
league = League(league_id=62847108, year=2024, debug = False)
teams = league.teams

In [3]:
1/2.5, 1/1.62

(0.4, 0.6172839506172839)

In [2]:
1/1.62 + 1/2.38

1.0374520178441746

In [1]:
1/1.53, 1/2.38

(0.6535947712418301, 0.42016806722689076)

## Docs

### Raw season

# Reading/Writing

In [537]:
RAW_SEASON_STATS = "raw_season_stats"
CLEAN_SEASON_STATS = "clean_season_stats"
CLEAN_PROJECTED_STATS = "clean_projected_season_stats"
SEASON_APPEARENCES = "season_appearences"
RAW_PLAYER_GAME_LOGS = "raw_player_logs"
LINEUPS = "lineups"

def write_csv(df, path):
    directory = os.path.dirname(path)
    os.makedirs(directory, exist_ok=True)
    df.to_csv(path + ".csv")

def write_json(d, path):
    directory = os.path.dirname(path)
    os.makedirs(directory, exist_ok=True)
    json.dump(d, open(path + ".json", "w"))

def try_read_json(path):
    try:
        if os.path.exists(path + ".json"):
            return json.load(open(path + ".json", "r"))
        return False
    except Exception as e:
        print(path)
        raise e

def read_raw_season_stats(year: int):
    return json.load(open(f"{year}/{RAW_SEASON_STATS}.json", "r"))

def write_raw_season_stats(year: int, stats):
    write_json(stats, f"{year}/{RAW_SEASON_STATS}")

def read_season_appearences(year):
    return pd.read_csv(f"{year}/{SEASON_APPEARENCES}/.csv")

def write_season_appearences(logs, year):
    write_csv(logs, f"{year}/{SEASON_APPEARENCES}")

def write_raw_game_logs(logs, player, year):
    write_json(logs, f"{year}/{RAW_PLAYER_GAME_LOGS}/{player}")

def try_read_raw_game_logs(player, year):
    return try_read_json(f"{year}/{RAW_PLAYER_GAME_LOGS}/{player}")

def read_clean_season_stats(year: int):
    return pd.read_csv(f"{year}/{CLEAN_SEASON_STATS}")

def write_clean_season_stats(year: int, stats):
    write_csv(stats, f"{year}/{CLEAN_SEASON_STATS}")

def read_clean_projected_season_stats(year: int):
    return pd.read_csv(f"{year}/{CLEAN_PROJECTED_STATS}")

def write_clean_projected_season_stats(year: int, stats):
    write_csv(stats, f"{year}/{CLEAN_PROJECTED_STATS}")

def write_lineups(lineups, year, week):
    write_csv(lineups, f"{year}/{LINEUPS}/{week}")

def read_lineups(year, week):
    pd.read_csv(f"{year}/{LINEUPS}/{week}.csv")

# Get all players and write to file

In [442]:
def get_players_batch(league: League, ids: int):
    players = []
    player_infos = league.player_info(playerId = ids)
    for player in player_infos:
        team = player.proTeam
        player_stats = dict()
        player_stats["id"] = player.playerId
        player_stats["name"] = player.name
        player_stats["position"] = player.position
        player_stats["stats"] = dict()
        for week in player.stats.keys():
            player.stats[week]["team"] = team
            #if (week < 0) and (week in schedule.keys()) and (team in schedule[week].keys()):
            #    player.stats[week]["opponent"] = schedule[week][team]
            #else:
            #    player.stats[week]["opponent"] = ""
            player_stats["stats"][week] = player.stats[week]
        players.append(player_stats)
    
    return players

def get_all_players(league: League, ids: List[int]):
    batch_size = 1000

    start = 0
    batch_size = 1_000

    players = []

    while start < len(ids):
        players += get_players_batch(league, ids[start:start+batch_size])
        start += batch_size
    
    write_raw_season_stats(league.year, players)

get_all_players(league, player_ids[:1002])

In [545]:
player_ids = [key for key in league.player_map if type(key) == int]

## Clean Stats

In [456]:
def create(player, actual):
    key = "breakdown" if actual else "projected_breakdown" # only exists for season total
    rows = []
    for week in player["stats"]:

        if not actual or week != "0":
             continue
         
        stats = dict()
        stats["id"] = player["id"]
        stats["week"] = week
        stats["position"] = player["position"]

        for k in player["stats"][week][key]:
                stats[k] = player["stats"][week][key][k]
                
        rows.append(stats)

    return rows


def clean_season_stats(season: int):

    actual_stats = read_raw_season_stats(2024)
    actual_rows = []
    projected_rows = []
    for player in actual_stats:
        actual_rows += create(player, True)
        projected_rows += create(player, False)


    actual_stats = pd.DataFrame(actual_rows)
    actual_stats.fillna(0, inplace=True)

    projected_stats = pd.DataFrame(actual_rows)
    projected_stats.fillna(0, inplace=True)

    write_clean_season_stats(season, actual_stats)
    write_clean_projected_season_stats(season, projected_stats)

clean_season_stats(2024)

## Calculate points

In [429]:
points_by_action = pd.read_csv("scoring_format.csv").set_index("lookupName")
points_by_action_series = points_by_action.points
points_by_action.head(1)

,Unnamed: 0,abbr,label,id,points,modified,WR,RB,TE,DE,LB,S,CB,DT,QB,K
lookupName,,,,,,,,,,,,,,,,
112,7.0,STF,Stuffs,112.0,0.5,True,False,False,False,True,True,True,True,True,False,False


In [430]:
def f(row):
    return (points_by_action.points * row[row.index.isin(points_by_action.index)].astype(float)).sum()

# stats["calculated"] = stats.apply(f, 1)

In [172]:
def get_stat_types(stats):
    stat_types = set()
    for player in stats:
        for week in player["stats"]:
            for stat_type in player["stats"][week]["breakdown"]:
                if not stat_type.isnumeric():
                    stat_types.add(stat_type)
    return stat_types

In [225]:
def parse_gamelog(gamelog: dict, season: int, player_id: int) -> dict:
    
    opponent = gamelog["opponent"]["id"]
    is_home = gamelog["homeTeamId"] != opponent
    team = gamelog["homeTeamId"] if is_home else gamelog["awayTeamId"]
    score = gamelog["homeTeamScore"] if is_home else gamelog["awayTeamScore"]
    opponent_score = gamelog["homeTeamScore"] if not is_home else gamelog["awayTeamScore"]
    date = gamelog["gameDate"]
    game_id = gamelog["id"]
    week = gamelog["week"]

    return {
        "PlayerId": player_id,
        "GameId": game_id,
        "Season": season,
        "Week": week,
        "Team": team,
        "Opponent": opponent,
        "Date": date,
        "TeamScore": score,
        "OpponentScore": opponent_score,
    }


def parse_season(season_logs, season, player):
    game_logs = []

    if "events" not in season_logs.keys():
        return game_logs
    
    for game_id in season_logs["events"].keys():
        game_logs.append(parse_gamelog(season_logs["events"][game_id], season, player))

    return game_logs


def get_season_appearences(player_ids, start_year, end_year):

    seasons = range(start_year, end_year+1)
    logs_by_season = {season: [] for season in seasons}

    for player in player_ids:
        time.sleep(0.5)
        print(player)
        for season in seasons:
            local_logs = try_read_raw_game_logs(player, season)
            if local_logs:
                season_logs = local_logs
            else:
                season_logs = requests.get(GAME_LOGS_LINK.format(player=player, year=season)).content
                season_logs = json.loads(season_logs)
                write_raw_game_logs(season_logs, player, season)
            logs_by_season[season] += parse_season(season_logs, season, player)

    for season in seasons:
        df = pd.DataFrame(logs_by_season[season])
        write_season_appearences(df, season)


logs = get_season_appearences([LAMAR, DICKER, 4242899], 2022, 2024)
        

3916387
4362081
4242899


In [11]:
def get_player_age_and_debut_year(id):
    url = f"http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/{id}"
    content = requests.get(url).content
    player_dict = json.loads(content)
    return player_dict["age"], player_dict["debutYear"]

get_player_age_and_debut_year(3045282)

(29, 2017)

In [540]:
def get_player_lineup_info(player):
    info = {
        #'acquisitionType': player.acquisitionType,
        'active_status': player.active_status,
        'avg_points': player.avg_points,
        #'eligibleSlots': player.eligibleSlots,
        #'game_date': player.game_date,
        #'game_played': player.game_played,
        'injured': player.injured,
        'injuryStatus': player.injuryStatus,
        'lineupSlot': player.lineupSlot,
        'name': player.name,
        'onTeamId': player.onTeamId,
        'on_bye_week': player.on_bye_week,
        #'percent_owned': player.percent_owned,
        #'percent_started': player.percent_started,
        'playerId': player.playerId,
        'points': player.points,
        #'points_breakdown': player.points_breakdown,
        #'posRank': player.posRank,
        'position': player.position,
        #'proTeam': player.proTeam,
        #'pro_opponent': player.pro_opponent,
        'pro_pos_rank': player.pro_pos_rank,
        #'projected_avg_points': player.projected_avg_points,
        #'projected_breakdown': player.projected_breakdown,
        #'projected_points': player.projected_points,
        #'projected_total_points': player.projected_total_points,
        #'schedule': player.schedule,
        'slot_position': player.slot_position,
        #'stats': player.stats,
        #'total_points': player.total_points,
    }

    return info

In [541]:
def get_match_lineup(match):
    return [get_player_lineup_info(p) for p in match.home_lineup + match.away_lineup]

def get_lineups(league, min_week, max_week):
    for week in range(min_week, max_week+1):
        player_info = []
        boxscores = league.box_scores(week)
        
        for match in boxscores:
            player_info += get_match_lineup(match)
        
        write_lineups(pd.DataFrame(player_info), league.year, week)

get_lineups(league, 1, 7)

In [544]:
teams = []
for team in league.teams:
    teams.append({"id": team.team_id, "name": team.team_name})

pd.DataFrame(teams).to_csv("lookup/teams.csv")